In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# So code is automatically reloaded when saved in different modules

In [2]:
import os
import pickle
import numpy as np
import tensorflow as tf
# from scipy import stats
import tensorflow as tf
# from sklearn.metrics import roc_auc_score
# import sklearn.metrics
from sklearn import metrics
import scipy
import matplotlib.pyplot as plt
import time
import logging
import sys
import time
from datetime import datetime
import csv
from collections import Counter, OrderedDict, defaultdict
import sys
import pprint
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
logger = logging.getLogger("hw3.q2")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

## File Paths


In [3]:
# data_train = "data/icd9NotesDataTable_train.csv"
# data_valid = "data/icd9NotesDataTable_valid.csv"
# data_train = "data/smallIcd9NotesDataTable_train.csv"
# data_valid = "data/smallIcd9NotesDataTable_valid.csv"
data_train = "data/smallIcd9NotesDataTable_train_2.csv"
data_valid = "data/smallIcd9NotesDataTable_train_2.csv"
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
output_path = 'results/model_description'
# log_output = output_path + "log"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
output_path = 'results/{}/{:%Y%m%d_%H%M%S}/".format(self.cell, datetime.now())'
log_output = output_path + "log"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50 # this should not be manually set and should instead come from the word
# vectors. Or maybe it's good that we have to set it so we know for sure the size
# of word vecs we're using
maxAllowedNoteLength = 500
max_grad_norm = 5
codeIdx = 9
textIdx = 6
num_layers = 4

In [4]:
modelRunOutputPath = input(prompt = 'Where should models and performances be saved to?')
if modelRunOutputPath == '':
    modelRunOutputPath = 'results/temp'
if not os.path.exists(modelRunOutputPath):
    os.makedirs(modelRunOutputPath)

Where should models and performances be saved to?


## Extract Data and Word Embeddings
You'll probably only need to work with xDev, yDev and xTrain, yTrain. The X matrices hold all word IDs in the order they appear in the note. yDev is a matrix of indicator vectors for icd9 presence

In [5]:
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx,
    helperLoadPath = 'results/temp/')
# embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
# helper.save(output_path)# token2id and max length saved to output_path
# handler = logging.FileHandler(log_output)
# handler.setLevel(logging.DEBUG)
# handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
# logging.getLogger().addHandler(handler)

In [6]:
print(helper.icdDict)
print(helper.tok2id)
print(helper.max_length)

{'18': 0, '456': 5, '123': 2, '4240': 1, '45': 4, '486': 3}
{'<s>': 11, '</s>': 10, 'horse': 3, 'CASE:Aa': 8, 'UUUNKKK': 9, 'CASE:AA': 5, 'deadpool': 4, 'dog': 1, 'CASE:aA': 7, 'CASE:aa': 6, 'cat': 2, 'welcometonightvale': 9}
4


In [7]:
print(helper.icdDict)
print(helper.tok2id)
print(helper.max_length)

{'18': 0, '456': 5, '123': 2, '4240': 1, '45': 4, '486': 3}
{'<s>': 11, '</s>': 10, 'horse': 3, 'CASE:Aa': 8, 'UUUNKKK': 9, 'CASE:AA': 5, 'deadpool': 4, 'dog': 1, 'CASE:aA': 7, 'CASE:aa': 6, 'cat': 2, 'welcometonightvale': 9}
4


In [8]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0

In [9]:
with open(os.path.join(modelRunOutputPath, 'runOutput.txt'), 'w') as f:
    with tf.Session() as session:
        new_saver = tf.train.import_meta_graph('results/temp/bestModel.meta')
        new_saver.restore(session, tf.train.latest_checkpoint('results/temp/'))
        x = tf.get_collection('x')[0]
        trueWordIdxs = tf.get_collection('trueWordIdxs')[0]
        outputKeepProb = tf.get_collection('outputKeepProb')[0]
        inputKeepProb = tf.get_collection('inputKeepProb')[0]
        pretrainedEmbeddings = tf.get_collection('pretrainedEmbeddings')[0]
        y_last = tf.get_collection('y_last')[0]
        batchSizeDev = 3295
        totalBatchesDev = (xDev.shape[0]//batchSizeDev)
        pred_y = np.full(shape = yDev.shape, fill_value = -1.0, dtype = np.float32)
        for b in range(totalBatchesDev + 1):
#                 print(b)
            offset = min((b * batchSizeDev), yDev.shape[0])
            devBatch_x = xDev[offset:(offset + batchSizeDev), :]
            devBatchTrueWordIdxs = lastTrueWordIdx_dev[offset:(offset + batchSizeDev)]
            pred_yBatch = session.run(y_last,feed_dict={x: devBatch_x,
                                                  trueWordIdxs:devBatchTrueWordIdxs,
                                                  outputKeepProb: 1,
                                                    inputKeepProb: 1}, ) # must be set to one for predictions.
            pred_y[offset:(offset + batchSizeDev), :] = pred_yBatch
        if (pred_y == -1).all():
            print('negative values exist. This means indexing is off in pred_yBatch')
            1/0
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
        print('here is the loss')
        print(validLoss)

here is the loss
0.955575


In [11]:
print(helper.icdDict)
print(helper.tok2id)
print(helper.max_length)

{'18': 0, '456': 5, '123': 2, '4240': 1, '45': 4, '486': 3}
{'<s>': 11, '</s>': 10, 'horse': 3, 'CASE:Aa': 8, 'UUUNKKK': 9, 'CASE:AA': 5, 'deadpool': 4, 'dog': 1, 'CASE:aA': 7, 'CASE:aa': 6, 'cat': 2, 'welcometonightvale': 9}
4


In [10]:
yDev

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])